#**Geração de Histogramas de Frequências de Palavras Visuais/Aurais**
Neste Notebook são geradas **caracterísiticas de médio nível semântico** a partir:
  
1. das características de baixo nível semântico previamente extraídas das imagens e do áudio de cada vídeo (SIFT e MFCC respectivamente);
2. dos dicionários de palavras visuais/aurais previamente computados.

Este é o segundo e último passo do processo de **Enriquecimento Semântico** adotado, o qual culmina em novas representações para cada vídeo da base na forma de histogramas de frequência: um de palavras visuais e outro de palavras aurais.  


#Preparação do ambiente do Notebook





**Instalando Dependências**

In [1]:
import numpy as np
import pickle
from scipy.spatial import distance


**Acesso ao Drive - para armazenamento de Arquivos**

O *pathname* precisa ser personalizado com o Drive do usuário deste Notebook. Atenção: alguns *pathnames* podem estar em Inglês enquanto outros em Português - verifique o seu caso. Crie um pasta no **seu** Drive onde os arquivos serão armazenados. Neste exemplo tal pasta é "MBA_Curso14_Dados".

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/MBA_Curso14_Dados/'

/content/drive/My Drive/MBA_Curso14_Dados


#<h1>Funções para a geração de histogramas de frequencia de palavras visuais/aurais</h1>

In [4]:
## ==========
# Função auxiliar. Normaliza os valores da lista de entrada. Usada para normalizar,
# posteriormente, se desejado, os histogramas computados.
## ==========
def normalizeFeatures(features):
    normalizedFeatures = []
    for i in range(len(features)):
        s = sum(features[i])
        if s == 0:
            normalizedFeatures.append(features[i])
        else:
            normalizedFeatures.append(features[i] / s)
    return normalizedFeatures
    
## ==========
# Função auxiliar. Retorna o índice do dicionário cujo conteúdo possui a menor 
# distância para um dado vetor de carcterísitcas ("feature") de entrada.
# Lembrando que cada posição do dicionário é um vetor de característica também.
## ==========
def getSimilar(feature, dicionario):
    dist = []
    
    #calcula a distância (similaridade) entre o vetor de entrada e todos os 
    #vetores (posições) do dicionário, armazenando as distância na lista "dist".
    for i in range(len(dicionario)): 
        #dist.append(np.linalg.norm(dicionario[i] - feature)) # dist Euclideana pela norma
        dist.append(distance.cosine(dicionario[i], feature))

    #retorna o índice de "dist" que contém a menor distância
    return dist.index(min(dist))

## ==========
# Calcula um histograma de frequencias de palavras (visuais ou aurais) para cada
# vídeo de entrada representado na lista "features". Armazena cada um dos 
# histogramas na lista "histograms".
## ==========
def getHistogram(features, dicionario):
    histograms = []

    for key in features:
      for bag in features[key]:
        arrayKeys = np.zeros(len(dicionario)) 
        if not(bag is None):
            #Calcula qual posição do dicionário possui a palavra mais similar ao 
            #vetor de features atual.
            for f in bag:
                index = getSimilar(f, dicionario)
                arrayKeys[index] += 1 
      histograms.append(arrayKeys)

    return histograms

#Script para a geração de histograma de palavras visuais
Será gerado um  histograma para cada vídeo da base, sendo que os histogramas serão armazenados em uma lista ("visualHistograms") e a lista será salva em arquivo. Cada histograma constitui uma **representação monomodal** (visual nesse caso) de um vídeo.

O arquivo poderá ser usado posteriormente para recuperar a lista e usar os histogramas que representam os vídeos em tarefas ou aplicações computacionais.

In [5]:
#Recuperando carcaterísticas visuais previamente extraídas e salvas em arquivo
with open('./features2/visual_features.arq', 'rb') as arq_visual_features:
    visualFeatures = pickle.load(arq_visual_features)

#Recuperando odicionário visual da base previamente calculado e salvo em arquivo
with open('./dicionarios2/visual_dic_100.arq', 'rb') as arq_visual_dic:
    visualDic = pickle.load(arq_visual_dic)

#Calculando os histogramas
visualHistograms = getHistogram(visualFeatures, visualDic)

#Normalizando os histogramas
normvisualHistograms = normalizeFeatures(visualHistograms)

#Salvado  os histogramas em arquivo
#A pasta "histogramas" deve existir dentro de "MBA_Curso14_Dados"
with open('./histogramas2/visual_histogram_100_cosine.arq', 'wb') as arq:
    pickle.dump(visualHistograms, arq)


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


#Script para a geração de histograma de palavras aurais
Será gerado um  histograma para cada vídeo da base, sendo que os histogramas serão armazenados em uma lista ("auralHistograms") e a lista será salva em arquivo. Cada histograma constitui uma **representação monomodal** (aural nesse caso) de um vídeo.

O arquivo poderá ser usado posteriormente para recuperar a lista e usar os histogramas que representam os vídeos em tarefas ou aplicações computacionais.

In [6]:
#Recuperando carcaterísticas aurais previamente extraídas e salvas em arquivo
with open('./features2/aural_features.arq', 'rb') as arq_aural_features:
  auralFeatures = pickle.load(arq_aural_features)

#Recuperando o dicionário aural da base previamente calculado e salvo em arquivo
with open('./dicionarios2/aural_dic_100.arq', 'rb') as arq_aural_dic:
    auralDic = pickle.load(arq_aural_dic)

#Calculando os histogramas
auralHistograms = getHistogram(auralFeatures, auralDic)

#Normalizando os histogramas
normauralHistograms = normalizeFeatures(auralHistograms)

#Salvado  os histogramas em arquivo
#A pasta "histogramas" deve existir dentro de "MBA_Curso14_Dados"
with open('./histogramas2/aural_histogram_100_cosine.arq', 'wb') as arq:
    pickle.dump(auralHistograms, arq)





